# Pandas Merge туториал

В реальных проектах данные обычно не хранятся в одной таблице. Вместо нее используется много маленьких. И на то есть несколько причин. С помощью нескольких таблиц данными легче управлять, проще избегать «многословия», можно экономить место на диске, а запросы к таблицам обрабатываются быстрее.

Суть в том, что при работе с данными довольно часто придется вытаскивать данные из двух и более разных страниц. Это делается с помощью merge.

In [2]:
import pandas as pd # импортируем библиотеку pandas

In [3]:
user_usage = pd.read_csv("user_usage.csv") # открываем три таблички
user_device = pd.read_csv("user_device.csv")
devices = pd.read_csv("android_devices.csv")

In [4]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [8]:
user_usage.use_id.nunique()

240

In [9]:
user_device.use_id.nunique()

272

In [11]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [14]:
devices.head(10)

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A
5,7Eleven,IN265,IN265,IN265
6,A.O.I. ELECTRONICS FACTORY,A.O.I.,TR10CS1_11,TR10CS1
7,AG Mobile,AG BOOST 2,BOOST2,E4010
8,AG Mobile,AG Flair,AG_Flair,Flair
9,AG Mobile,AG Go Tab Access 2,AG_Go_Tab_Access_2,AG_Go_Tab_Access_2


## Объединение по id пользователя

* user_usage — это левая таблица, которая является основным объектом для слияния.
* user_device[['use_id', 'platform', 'device']] — это правая таблица, из которой берутся только столбцы use_id, platform и device. 
* on='use_id' указывает столбец, на основе которого будет происходить объединение таблиц. В данном случае это столбец use_id, который должен присутствовать в обеих таблицах.
После выполнения слияния результат сохраняется в переменной result.

In [15]:
result = pd.merge(user_usage, # левая таблица
                 user_device[['use_id', 'platform', 'device']], # из правой берем три столбца
                 on='use_id') # на основании чего соединяем
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


Посмотрим на размерности получившихся таблиц:

In [16]:
print("user_usage размерности: {}".format(user_usage.shape))
print("user_device размерности: {}".format(user_device[['use_id', 'platform', 'device']].shape))


user_usage размерности: (240, 4)
user_device размерности: (272, 3)


In [17]:
result.shape

(159, 6)

In [18]:
user_usage['use_id'].isin(user_device['use_id']).value_counts()

use_id
True     159
False     81
Name: count, dtype: int64

Базовый метод merge довольно прост. Но иногда к нему нужно добавить несколько параметров.

Один из самых важных вопросов — как именно нужно объединять эти таблицы.Есть 4 варианта:

![](https://deepage.net/img/pandas/merge/join-type.jpg)

## Left merge 

In [19]:
result = pd.merge(user_usage, # левая таблица
                 user_device[['use_id', 'platform', 'device']], # из правой три столбца
                 on='use_id', how='left') # на основании чего соединяем
print("user_usage размерности: {}".format(user_usage.shape)) # размерность таблицы user_usage
print("result размерности: {}".format(result.shape)) # размерность таблицы result
print("Всего {} пропущенных значений.".format(
        result['device'].isnull().sum()))  # количество пропусков в столбце device для результата

user_usage размерности: (240, 4)
result размерности: (240, 6)
Всего 81 пропущенных значений.


In [20]:
result['device'].isnull().sum()

81

In [21]:
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


## Right merge 


In [22]:
result = pd.merge(user_usage, # левая таблица
                 user_device[['use_id', 'platform', 'device']], # из правой три столбца
                 on='use_id', how='right') # на основании чего соединяем
print("user_device размерностиs: {}".format(user_device.shape))
print("result размерности: {}".format(result.shape))
print("Всего {} пропущенных значений в колонке 'monthly_mb' .".format(
        result['monthly_mb'].isnull().sum()))
print("Всего {} пропущенных значений в колонке 'platform'.".format(
        result['platform'].isnull().sum()))

user_device размерностиs: (272, 6)
result размерности: (272, 6)
Всего 113 пропущенных значений в колонке 'monthly_mb' .
Всего 0 пропущенных значений в колонке 'platform'.


## Outer merge 

In [23]:
pd.concat([user_usage['use_id'], user_device['use_id']]).nunique()

353

In [24]:
print("Всего {} уникальных значений для use_id.".format(
        pd.concat([user_usage['use_id'], user_device['use_id']]).unique().shape[0]))
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', how='outer', indicator=True)

print("Получаем в таблице {} строк.".format(result.shape))


Всего 353 уникальных значений для use_id.
Получаем в таблице (353, 7) строк.


In [25]:
result

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,_merge
0,NaN,NaN,NaN,22782,ios,"iPhone7,2",right_only
1,NaN,NaN,NaN,22783,android,Nexus 5,right_only
2,NaN,NaN,NaN,22784,android,SM-G903F,right_only
3,NaN,NaN,NaN,22785,ios,"iPhone7,2",right_only
4,NaN,NaN,NaN,22786,android,ONE E1003,right_only
...,...,...,...,...,...,...,...
348,260.66,68.44,896.96,25008,NaN,NaN,left_only
349,97.12,36.50,2815.00,25040,NaN,NaN,left_only
350,355.93,12.37,6828.09,25046,NaN,NaN,left_only
351,632.06,120.46,1453.16,25058,NaN,NaN,left_only


In [26]:
devices

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A
...,...,...,...,...
14541,pendo,PNDPP44QC10,PNDPP44QC10,PNDPP44QC10
14542,pendo,PNDPP44QC7,PNDPP44QC7,PNDPP44QC7
14543,sugar_aums,QPOINT,QPI-1,QPI-1
14544,tecmobile,OmnisOne,OmnisOne,Omnis One


## Final merge


In [27]:

result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id',
                 how='left')


devices.rename(columns={"Retail Branding": "manufacturer"}, inplace=True) # изменение названий столбцов
result = pd.merge(result, 
                  devices[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')

result.head()
                  
                               

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


In [28]:
result.drop('Model', axis = 1, inplace = True)

## Вычисление итоговых статистик

In [29]:
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung
3,94.46,35.17,519.12,22790,android,D2303,Sony
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung


In [30]:
result.groupby("manufacturer").agg({
        "outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "max",
        "use_id": "count"
    })

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,15573.33,44
Huawei,81.526667,9.500000,3114.67,3
LGE,111.530000,12.760000,1557.33,2
Lava,60.650000,261.900000,12458.67,2
Lenovo,215.920000,12.930000,1557.33,2
Motorola,95.127500,65.666250,5191.12,16
OnePlus,354.855000,48.330000,15573.33,6
Samsung,191.010093,92.390463,31146.67,108
Sony,177.315625,40.176250,7267.55,16
